# Avaliação do Impacto de Tweets sobre o Valor de Ações

## Importar Packages

In [ ]:
import pandas as pd
from google.colab import drive
from datetime import datetime
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importar Datasets

In [ ]:
stocks=pd.read_csv('/content/drive/MyDrive/Datasets/top5_prices_v2.csv')

In [ ]:
orig_nasdaq=pd.read_csv('/content/drive/MyDrive/Datasets/projetoIPAI/nasdaq_historical_prices_daily.csv')
comp_tweet=pd.read_csv('//content/drive/MyDrive/Datasets/projetoIPAI/Company_Tweet.csv')
tweets=pd.read_csv('//content/drive/MyDrive/Datasets/projetoIPAI/Tweet.csv')
company=pd.read_csv('//content/drive/MyDrive/Datasets/projetoIPAI/Company.csv')

In [ ]:
orig_nasdaq.head(3)

,ticker,date,open,high,low,close,volume
0,AAPL,2020-02-06,322.57,325.22,320.2648,325.21,26214054
1,AAPL,2020-02-05,323.52,324.76,318.9500,321.45,29384908
2,AAPL,2020-02-04,315.31,319.64,313.6345,318.85,34154134


In [ ]:
comp_tweet.head(3)

,tweet_id,ticker_symbol
0,550803612197457920,AAPL
1,550803610825928706,AAPL
2,550803225113157632,AAPL


In [ ]:
tweets.head(3) # post_date em Unix TimeStamp!

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,1420070457,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,1420070496,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,1420070510,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0


In [ ]:
company.head(3) # Google tem 2 valores de ticker_symbol

,ticker_symbol,company_name
0,AAPL,apple
1,GOOG,Google Inc
2,GOOGL,Google Inc


# Dataset A

## Selecionar apenas dados de AAPL, AMZN, GOOGL, MSFT e TSLA a partir de 2015

In [ ]:
def ticker15(ticker, df):
    '''Recebe uma lista de tickers e um dataframe de ações de empresas
     e cria um dataframe de preços de ações dessas empresas a partir de 2015'''
    ticker_list=[]
    for i in df.index:
        if df['ticker'][i] in ticker:
            ticker_list.append(i)
            
    temp_df=df.iloc[ticker_list]
    df15=temp_df[temp_df.date>'2014-12-31'].sort_values(by='date').reset_index(drop=True)
    
    return df15

In [ ]:
tickers=['AAPL','TSLA','MSFT','AMZN','GOOGL']
stocks=ticker15(tickers, orig_nasdaq).sort_values(by=['ticker','date']).reset_index(drop=True)
stocks.head(5)

,ticker,date,open,high,low,close,volume
0,AAPL,2015-01-02,111.39,111.44,107.350,109.33,53204626
1,AAPL,2015-01-05,108.29,108.65,105.410,106.25,64285491
2,AAPL,2015-01-06,106.54,107.43,104.630,106.26,65797116
3,AAPL,2015-01-07,107.20,108.20,106.695,107.75,40105934
4,AAPL,2015-01-08,109.23,112.15,108.700,111.89,59364547


In [ ]:
stocks.ticker.unique()

array(['AAPL', 'AMZN', 'GOOGL', 'MSFT', 'TSLA'], dtype=object)

In [ ]:
# verificar se ticker GOOG existe no dataset de ações
if 'GOOG' in orig_nasdaq.ticker.unique():
  print('PROBLEMA!')
else:
  print('OK!')

OK!


# Criar novas variáveis `diff_openclose` e `diff_highlow` com a diferença entre esses valores

In [ ]:
highlow=[]
for i in range(len(stocks)):
  highlow.append(round(stocks.high[i]-stocks.low[i],2))
highlow[:5]

[4.09, 3.24, 2.8, 1.51, 3.45]

In [ ]:
openclose=[]
for i in range(len(stocks)):
  openclose.append(round(stocks.open[i]-stocks.close[i],2))
openclose[:5]

[2.06, 2.04, 0.28, -0.55, -2.66]

In [ ]:
stocks['diff_openclose']=openclose
stocks['diff_highlow']=highlow
stocks_2=stocks.drop(['high','low','open','close'], axis=1)
#stocks_2=stocks_2[['ticker', 'date', 'volume', 'diff_openclose', 'diff_highlow']]
stocks_2.head(5)

,ticker,date,volume,diff_openclose,diff_highlow
0,AAPL,2015-01-02,53204626,2.06,4.09
1,AAPL,2015-01-05,64285491,2.04,3.24
2,AAPL,2015-01-06,65797116,0.28,2.80
3,AAPL,2015-01-07,40105934,-0.55,1.51
4,AAPL,2015-01-08,59364547,-2.66,3.45


# Dataset B - Tweets

# Converter `post_date` de Unix TimeStamp para date

In [ ]:
new_time=[]
for ts in tweets['post_date']:
  new_time.append(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S'))

only_date=[]
for datatempo in new_time:
  only_date.append(datatempo[:10])

only_date[:5]

['2015-01-01', '2015-01-01', '2015-01-01', '2015-01-01', '2015-01-01']

In [ ]:
tweets['new_date']=only_date
tweets_2=tweets.drop('post_date', axis=1)
tweets_2=tweets_2[['tweet_id', 'writer', 'new_date', 'body', 'comment_num', 
               'retweet_num', 'like_num']]

In [ ]:
tweets_2.rename(columns={'new_date':'post_date'}, inplace=True)
tweets_2.head(3)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,2015-01-01,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,2015-01-01,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0


# Transformar GOOG em GOOGL em `company_tweet` 

In [ ]:
comp_tweet.ticker_symbol.unique()

array(['AAPL', 'GOOG', 'GOOGL', 'AMZN', 'MSFT', 'TSLA'], dtype=object)

In [ ]:
new_tickers=[]
for i in comp_tweet.ticker_symbol:
  if i == 'GOOG':
    new_tickers.append('GOOGL')
  else:
    new_tickers.append(i)

In [ ]:
comp_tweet['new_ticker']=new_tickers
#tweets=tweets.drop('post_date', axis=1)
comp_tweet_2=comp_tweet.drop('ticker_symbol', axis=1)
comp_tweet_2=comp_tweet_2.rename(columns={'new_ticker':'ticker'})
comp_tweet_2.ticker.unique()

array(['AAPL', 'GOOGL', 'AMZN', 'MSFT', 'TSLA'], dtype=object)

## Substituir valores NA em writer por "unknown_writer"

In [ ]:
tweets_2.isnull().sum() # só writer tem valores NA

tweet_id           0
writer         47273
post_date          0
body               0
comment_num        0
retweet_num        0
like_num           0
dtype: int64

In [ ]:
tweets_2=tweets_2.fillna('unknown_writer')
tweets_2.head(3)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,2015-01-01,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,2015-01-01,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0


In [ ]:
tweets_2.isnull().sum() # só writer tem valores NA

tweet_id       0
writer         0
post_date      0
body           0
comment_num    0
retweet_num    0
like_num       0
dtype: int64

# Datasets finais

## Tweet

In [ ]:
tweets_2=tweets_2.sort_values(by=['post_date','tweet_id']).reset_index(drop=True)
tweets_2.head(3)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
0,550441509175443456,VisualStockRSRC,2015-01-01,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1
1,550441672312512512,KeralaGuy77,2015-01-01,Insanity of today weirdo massive selling. $aap...,0,0,0
2,550441732014223360,DozenStocks,2015-01-01,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0


In [ ]:
tweets_2.to_csv('/content/drive/MyDrive/Datasets/projetoIPAI/tweet_final.csv')

## Mentions

In [ ]:
mentions=comp_tweet_2.sort_values(by='tweet_id').reset_index(drop=True)
mentions.head(3)

,tweet_id,ticker
0,550441509175443456,AAPL
1,550441672312512512,AAPL
2,550441732014223360,AMZN


In [ ]:
mentions.to_csv('/content/drive/MyDrive/Datasets/projetoIPAI/mentions_final.csv')

## Company

In [ ]:
company_2=company.drop(1).drop('company_name', axis=1)
company_2['name']=['Apple', 'Google', 'Amazon', 'Tesla', 'Microsoft']
company_2=company_2.rename(columns={'ticker_symbol':'ticker'})
company_2=company_2.sort_values(by='ticker')
company_2

,ticker,name
0,AAPL,Apple
3,AMZN,Amazon
2,GOOGL,Google
5,MSFT,Microsoft
4,TSLA,Tesla


In [ ]:
company_2.to_csv('/content/drive/MyDrive/Datasets/projetoIPAI/company_final.csv')

## Stock

In [ ]:
stocks_2.head(5)

,ticker,date,volume,diff_openclose,diff_highlow
0,AAPL,2015-01-02,53204626,2.06,4.09
1,AAPL,2015-01-05,64285491,2.04,3.24
2,AAPL,2015-01-06,65797116,0.28,2.80
3,AAPL,2015-01-07,40105934,-0.55,1.51
4,AAPL,2015-01-08,59364547,-2.66,3.45


In [ ]:
stocks_2.to_csv('/content/drive/MyDrive/Datasets/projetoIPAI/stock_final.csv')